In [ ]:
# %%

import pandas as pd

from source.utils import FREQ_DIR, RESULT_DIR, UCS_DIR, confirm_dir
from source.utils.associate import (BINARY_ASSOC_ARGS, add_extra_am,
                                    associate_ucs, confirm_basic_ucs)
from source.utils.associate import convert_ucs_to_csv as ucs2csv
from source.utils.associate import get_associations_csv as init_am, AM_DF_DIR
from source.utils.associate import manipulate_ucs, seek_readable_ucs, adjust_assoc_columns
pd.set_option('display.float_format', '{:,.2f}'.format)

# %% [markdown]

 set parameters:
 - `UNIT` options:
   - `'Bigr'`
   - `'Adv'`
   - `'Adj'`
 - `PAT_DIR` options:
   - any directory in `results/freq_out/` containing a `ucs_format/` subdir with a `Trig*.tsv` frequency table
   - developed with the following in mind:
     - `POSmirror`
     - `NEGmirror`
     - `ANYmirror` (which has been populated by simple concatenation in the shell)

In [ ]:
UNIT = 'Adv'
# UNIT = 'Bigr' 
# UNIT = 'Adv'
# PAT_DIR = 'POSmirror'
# PAT_DIR = 'NEGmirror'
PAT_DIR = 'ANYmirror'
FRQ_FLOOR = 10
TRIG_TSV = FREQ_DIR.joinpath(
    f'{PAT_DIR}/ucs_format/Trig{UNIT}_frq-thrMIN-7.35f.tsv')
FOCUS = ['f', 'unexpected_f',
         'conservative_log_ratio',
         'am_p1_given2', 'am_p2_given1',
         'am_p1_given2_simple', 'am_p2_given1_simple',
         'am_log_likelihood',
         'mutual_information', 'am_odds_ratio_disc', 't_score',
         'N', 'f1', 'f2', 'E11',
         'l1', 'l2']

In [ ]:
# %%

TRIGGER_POLARITY = {
    'positive': {'all',
                 'always',
                 'both',
                 'either',
                 'every',
                 'everybody',
                 'everyone',
                 'everything',
                 'many',
                 'often',
                 'or',
                 'some',
                 'somebody',
                 'someone',
                 'something',
                 'somethings',
                 'sometimes'},
    'negative': {'barely',
                 'hardly',
                 'neither',
                 'never',
                 'no',
                 'nobody',
                 'none',
                 'nor',
                 'nothing',
                 'seldom',
                 'rarely',
                 'scarcely'},

}

In [ ]:
# %%

TRIGGER_QUANT = {
    'existential': {
        'some',
        'somebody',
        'someone',
        'something',
        'somethings',
        'either',
     'or',
        'sometimes'
    },
    'universal': {
        'all', 'every',
        'always',
        'everybody',
        'everyone',
        'both',
        'everything',
    },
    'not_exist': {
        'neither',
        'never',
        'no',
        'nobody',
        'none',
        'nor',
        'nothing',
    },
    'hedged_not_exist': {
        'barely',
        'hardly',
        'rarely',
        'scarcely',
        'seldom',
        'few'
    },
    'hedged_universal':{
     'many',
     'often',
     }
}

In [ ]:
# %%

def invert_set_dict(d: dict):
    return {v: k for k in d for v in d[k]}

# %% [markdown]

## 1. Run `seek_readable_ucs()` to generate consistent output path

In [ ]:
readable = seek_readable_ucs(min_freq=FRQ_FLOOR,
                             ucs_subdir='trigger_eval',
                             contained_counts_path=TRIG_TSV)
print(readable.relative_to(RESULT_DIR))

    > seeking `trigger_eval/ANYmirror/readable/TrigAdv_frq-thrMIN-7.35f_min10x*` frequency data and initial associations...
ucs/trigger_eval/ANYmirror/readable/TrigAdv_frq-thrMIN-7.35f_min10x.rsort-view_am-only.csv


# %% [markdown]

 Snippet of frequencies to be analyzed (`TRIG_TSV`)

In [ ]:
! head -5 {TRIG_TSV} | column -t

64024  something  more
47547  all        more
45543  or         even
45382  all        too
40833  all        very


# %% [markdown]

## 2. Run `confirm_basic_ucs()`

In [ ]:
basic_ucs_path = readable.parent.parent.joinpath(
    readable.name.replace('.rsort-view_am-only.txt', '.ds.gz'))
print(f'Creating initial UCS table: `{basic_ucs_path.relative_to(RESULT_DIR)}')

basic_ucs_path = confirm_basic_ucs(
    basic_ucs_path,
    freq_floor=FRQ_FLOOR,
    contained_counts_path=TRIG_TSV)

Creating initial UCS table: `ucs/trigger_eval/ANYmirror/TrigAdv_frq-thrMIN-7.35f_min10x.rsort-view_am-only.csv
+ existing UCS table found ✓


# %% [markdown]

 Excerpt of initial UCS table

In [ ]:
init_readable = UCS_DIR.joinpath(
    f'trigger_eval/{PAT_DIR}/readable'
).joinpath(f'{TRIG_TSV.name.replace(".tsv","")}_min{FRQ_FLOOR}x.init.txt')
! head -7 {init_readable}

        l1  l2                        f      f2      f1        N  
----------  --------------------  -----  ------  ------  -------  
        or  more                  34344  285602  326614  1761853  
 something  more                  64024  285602  318343  1761853  
       all  more                  47547  285602  249279  1761853  
      some  more                  15949  285602  114929  1761853  
     never  more                  20655  285602  109723  1761853  


# %% [markdown]

## 3. Run `associate_ucs()`

In [ ]:
associate_ucs(basic_ucs_path)

transform_ucs_log = f'/share/compling/projects/sanpi/logs/associate/ucs//ucs-{PAT_DIR}_Trig{UNIT}_frq-thrMIN-7-35f_min{FRQ_FLOOR}x*.log'
! head -15 `ls -t1 {transform_ucs_log} | head -1`
! echo '...'
! tail -2 `ls -t1 {transform_ucs_log} | head -1`


Calculating UCS associations...

```
bash /share/compling/projects/sanpi/script/transform_ucs.sh /share/compling/projects/sanpi/results/ucs/trigger_eval/ANYmirror/TrigAdv_frq-thrMIN-7.35f_min10x.rsort-view_am-only.csv
> log will be saved to: /share/compling/projects/sanpi/logs/associate/ucs//ucs-ANYmirror_TrigAdv_frq-thrMIN-7-35f_min10x.2024-05-25_2042.log
...
```

+ time elapsed → 00:00:13.496
# Manipulating TrigAdv_frq-thrMIN-7-35f_min10x ucs table
path to this script: /share/compling/projects/sanpi/script/transform_ucs.sh
Sat May 25 20:42:10 EDT 2024
(TMP: /share/compling/projects/sanpi/results/ucs/trigger_eval/ANYmirror/tmp/tmp_ANYmirror-20240525-204210.TrigAdv_frq-thrMIN-7-35f_min10x)
Caught a SIGPIPE at /usr/share/perl/5.30/Pod/Perldoc.pm line 1910.
Can't close STDOUT: Broken pipe at (eval 44) line 1.
END failed--call queue aborted.
## Initial Contingency Info

DATA SET FILE:  /share/compling/projects/sanpi/results/ucs/trigger_eval/ANYmirror/TrigAdv_frq-thrMIN-7.35f_min10x.rsort

# %% [markdown]

### Define dictionary containing relevant vocab sizes
 !!! Warning This is a `#HACK`: \
     Rather than developing a command/code to retrieve the vocab sizes programmatically,
     I simply copied the values given in the log output of `transform_usc.sh`
     for each `PAT_DIR`+`UNIT` combination

 |      |   ANYmirror |   NEGmirror |   POSmirror |
 |:-----|------------:|------------:|------------:|
 | Bigr |     395,338 |      64,644 |     330,694 |
 | Adv  |      23,125 |       5,004 |      18,121 |
 | Adj  |      83,422 |      21,562 |      61,860 |

In [ ]:
VOCABS = {'ANYmirror': {'Adv': 23125, 'Bigr': 395338, 'Adj': 83422},
          'NEGmirror': {'Adv': 5004, 'Bigr': 64644, 'Adj': 21562},
          'POSmirror': {'Adv': 18121, 'Bigr': 330694, 'Adj': 61860}
          }  # HACK
VOCAB = VOCABS[PAT_DIR][UNIT]
pd.DataFrame(VOCABS)

,ANYmirror,NEGmirror,POSmirror
Adv,23125,5004,18121
Bigr,395338,64644,330694
Adj,83422,21562,61860


# %% [markdown]

## 4. Run `ucs_to_csv()` to convert `ucs/[PAT_DIR]/readable/*.txt` to format that `pandas` can parse as a dataframe

In [ ]:
! head -5 {readable}
csv_path = ucs2csv(readable)
print(f'CSV: `{csv_path.relative_to(RESULT_DIR)}`')

        l1  l2                        f           E11  am.log.likelihood  am.odds.ratio.disc   am.p1.given2    am.p2.given1  am.p1.given2.simple  am.p2.given1.simple      f1      f2        N  
----------  --------------------  -----  ------------  -----------------  ------------------  -------------  --------------  -------------------  -------------------  ------  ------  -------  
        or  more                  34344  52945.172854      -10447.028600        -0.256703631  -0.0777299789  -0.06991187911         0.1202512587        0.10515164690  326614  285602  1761853  
 something  more                  64024  51604.417330        4160.189636         0.142486132   0.0518985499   0.04761694970         0.2241720996        0.20111640589  318343  285602  1761853  
       all  more                  47547  40408.922287        1689.350852         0.101020081   0.0298283680   0.03335405322         0.1664799266        0.19073808865  249279  285602  1761853  
UCS table text converted & saved as

# %% [markdown]

##

In [ ]:
trig_amdf = pd.read_csv(csv_path).convert_dtypes()
trig_amdf

,l1,l2,f,E11,am_log_likelihood,am_odds_ratio_disc,am_p1_given2,am_p2_given1,am_p1_given2_simple,am_p2_given1_simple,f1,f2,N
0,or,more,34344,"52,945.17","-10,447.03",-0.26,-0.08,-0.07,0.12,0.11,326614,285602,1761853
1,something,more,64024,"51,604.42","4,160.19",0.14,0.05,0.05,0.22,0.20,318343,285602,1761853
2,all,more,47547,"40,408.92","1,689.35",0.10,0.03,0.03,0.17,0.19,249279,285602,1761853
3,some,more,15949,"18,630.36",-511.92,-0.08,-0.01,-0.02,0.06,0.14,114929,285602,1761853
4,never,more,20655,"17,786.45",566.82,0.08,0.01,0.03,0.07,0.19,109723,285602,1761853
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4884,often,mordantly,11,0.84,45.56,1.58,0.64,0.00,0.69,0.00,92233,16,1761853
4885,everything,humanely,11,0.34,66.33,2.05,0.71,0.00,0.73,0.00,39516,15,1761853
4886,something,unthinkably,10,2.35,21.37,1.13,0.59,0.00,0.77,0.00,318343,13,1761853
4887,something,primally,13,2.35,44.49,2.09,0.82,0.00,1.00,0.00,318343,13,1761853


In [ ]:
# %%

trig_amdf['key'] = (trig_amdf.l1 + '~' +
                    trig_amdf.l2).astype('string')
trig_amdf = trig_amdf.set_index('key')
trig_amdf

,l1,l2,f,E11,am_log_likelihood,am_odds_ratio_disc,am_p1_given2,am_p2_given1,am_p1_given2_simple,am_p2_given1_simple,f1,f2,N
key,,,,,,,,,,,,,
or~more,or,more,34344,"52,945.17","-10,447.03",-0.26,-0.08,-0.07,0.12,0.11,326614,285602,1761853
something~more,something,more,64024,"51,604.42","4,160.19",0.14,0.05,0.05,0.22,0.20,318343,285602,1761853
all~more,all,more,47547,"40,408.92","1,689.35",0.10,0.03,0.03,0.17,0.19,249279,285602,1761853
some~more,some,more,15949,"18,630.36",-511.92,-0.08,-0.01,-0.02,0.06,0.14,114929,285602,1761853
never~more,never,more,20655,"17,786.45",566.82,0.08,0.01,0.03,0.07,0.19,109723,285602,1761853
...,...,...,...,...,...,...,...,...,...,...,...,...,...
often~mordantly,often,mordantly,11,0.84,45.56,1.58,0.64,0.00,0.69,0.00,92233,16,1761853
everything~humanely,everything,humanely,11,0.34,66.33,2.05,0.71,0.00,0.73,0.00,39516,15,1761853
something~unthinkably,something,unthinkably,10,2.35,21.37,1.13,0.59,0.00,0.77,0.00,318343,13,1761853


# %% [markdown]

## 6. Save to `./results/assoc_df/`

In [ ]:
df_csv_path = AM_DF_DIR.joinpath(
    str(csv_path.relative_to(UCS_DIR))
    .replace('/readable', '')
    .replace('.rsort-view_am-only', ''))

if not df_csv_path.is_file():
    confirm_dir(df_csv_path.parent)
    trig_amdf.to_csv(df_csv_path)

df_pkl_path = df_csv_path.with_suffix('.pkl.gz')
if not df_pkl_path.is_file():
    trig_amdf.to_pickle(df_csv_path.with_suffix('.pkl.gz'))

# %% [markdown]

## 7. Add additional AM via `add_extra_am()`

In [ ]:
ex_trig_amdf = add_extra_am(df=trig_amdf,
                            verbose=True,
                            vocab=VOCAB,
                            metrics=['t_score', 'mutual_information']
                            ).convert_dtypes()


Preview of Extended Measures (rounded)

| key            |   t_score |   mutual_information |   deltaP_min |   deltaP_max |   deltaP_max_abs |   deltaP_product |   unexpected_f |   unexpected_ratio |
|:---------------|----------:|---------------------:|-------------:|-------------:|-----------------:|-----------------:|---------------:|-------------------:|
| or~more        |   -100.37 |                -0.19 |        -0.08 |        -0.07 |             0.08 |             0.01 |     -18,601.17 |              -0.54 |
| something~more |     49.08 |                 0.09 |         0.05 |         0.05 |             0.05 |             0.00 |      12,419.58 |               0.19 |
| all~more       |     32.74 |                 0.07 |         0.03 |         0.03 |             0.03 |             0.00 |       7,138.08 |               0.15 |
| some~more      |    -21.23 |                -0.07 |        -0.02 |        -0.01 |             0.02 |             0.00 |      -2,681.36 |              -0.17 |

#%% [markdown]

 Add trigger features as columns: polarity and quantification type

In [ ]:
def add_feature(triggers:pd.Series, 
                group_dict: dict) -> pd.Series:
    assign_dict = invert_set_dict(group_dict)
    return triggers.apply(lambda x: assign_dict[x] if x in assign_dict.keys() else '')

ex_trig_amdf['polarity'] = add_feature(ex_trig_amdf.l1, TRIGGER_POLARITY)
ex_trig_amdf['quant'] = add_feature(ex_trig_amdf.l1, TRIGGER_QUANT)

# %% [markdown]

 Save extended AM tables to `extra/` subdirectory if not already saved

In [ ]:
df_extra_csv = df_csv_path.parent / 'extra' / \
    df_csv_path.name.replace('.csv', '_extra.csv')
print(df_extra_csv)
if not df_extra_csv.is_file():
    confirm_dir(df_extra_csv.parent)
    ex_trig_amdf.to_csv(df_extra_csv)

df_extra_pkl = df_extra_csv.with_suffix('.pkl.gz')
if not df_extra_pkl.is_file():
    ex_trig_amdf.to_pickle(df_extra_pkl)

/share/compling/projects/sanpi/results/assoc_df/trigger_eval/ANYmirror/extra/TrigAdv_frq-thrMIN-7.35f_min10x_extra.csv


In [ ]:
# %%

ex_trig_full = ex_trig_amdf.copy()
ex_trig_abbr = adjust_assoc_columns(
    ex_trig_amdf[[c for c in ['polarity', 'quant'] + FOCUS if c in ex_trig_amdf.columns]]).sort_values('LRC', ascending=False)
cols = ex_trig_abbr.columns

# %% [markdown]

 Define lexical items with given lean shown in binary environment evaluation

In [ ]:
pos_prone = {
    'Adj': [
        'unrelated',
        'unable',
        'akin',
        'larger',
        'different',
        'familiar',
        'similar',
        'likely', 
        'brief', 
        'unaware'
    ],
    'Adv': [
        'slightly', 
        'definitely',
        'utterly',
        # LRC top
        'pretty', 
        'rather', 
        'plain', 
        'fairly', 
        'somewhat', 
        'otherwise', 
        'downright',
        'relatively', 
        # G2 top
        'very', 
        'even', 
        'just', 
        # dP1 top (and odds ratio disc)
        'plain', 
        'maybe'
    ], 
    'Bigr': [
        # G2 top
        'completely_different', 
        'too_familiar',
        'even_better',
        # dP1 top
        'quite_different', 
        'too_real', 
        'well_aware', 
        # LRC top
        'too_common', 
        'entirely_different'
        ]}
neg_prone = {
    'Bigr': [
        # LRC top
        'quite_sure',
        'really_sure', 
        'too_early', 
        'too_pleased', 
        'too_fancy',
        # dP1 top
        'entirely_sure', 
        'ever_easy', 
        'ever_perfect', 
        'particularly_surprising',
        'particularly_new',
        # G2 top
        'too_late', 
        'more_important',
        'so_easy', 
        'as_good', 
        'too_old'
        ], 
    'Adv': [
        'yet',
        #LRC top
        'ever',
        'any', 
        'longer', 
        'necessarily',
        'that',
        #dP1 top
        'before', 
        'wise',  #? How is this used as an adverb?
        'earthly',
        'remotely',
        'exactly',
        # G2 top
        'particularly', 
        'too',
        'inherently'
        ],
    'Adj': [
        # LRC top
        'early', 
        'late', 
        'fancy', 
        'alone', 
        'sure',
        # dP1 top
        'shabby',
        'demoralizing',
        'alone',
        'aggravating',
        'groundbreaking',
        'eventful',
        #G2 top
        'important', 
        'frustrating',
        'evident', 
        'certain'
        ]
    }
def sort_prone_by_f2(prone_list, amdf):
    return amdf.copy().loc[amdf.l2.isin(prone_list), ['f2','l2']].drop_duplicates().reset_index(drop=True).set_index('l2').round(1).sort_values(['f2'], ascending=False).index.to_list()
pos_prone[UNIT] = sort_prone_by_f2(pos_prone[UNIT], ex_trig_abbr)
neg_prone[UNIT] = sort_prone_by_f2(neg_prone[UNIT], ex_trig_abbr)
pos_prone[UNIT]

['very',
 'even',
 'just',
 'pretty',
 'rather',
 'slightly',
 'otherwise',
 'fairly',
 'relatively',
 'plain',
 'downright',
 'somewhat',
 'maybe',
 'utterly',
 'definitely']

# %% [markdown]

## Strongest associations for each polarity by metric

In [ ]:
def show_metric_top(amdf: pd.DataFrame,
                    metric: str,
                    k=5, cols=[None]):
    if not any(cols):
        cols = amdf.columns
    return (pd.concat((polar_df.nlargest(k, metric)
                       for pol, polar_df in amdf.groupby('polarity')))
            .loc[:, [metric] + cols[cols != metric].to_list()]
            .reset_index(drop=True).set_index(['polarity', 'l1', 'l2'])
            )
def update_prone(exdf, pos_prone, neg_prone) -> dict: 
    prone_dict = {'positive':pos_prone, 
                  'negative': neg_prone}
    for polarity, prone in prone_dict.items(): 
        for topw in exdf.loc[polarity, :].reset_index().l2.head(2).squeeze(): 
            if topw not in prone[UNIT]: 
                prone[UNIT].append(topw)
        prone_dict[polarity] = prone
    return prone_dict['positive'], prone_dict['negative']

# %% [markdown]

 Top consevative log ratio $LRC$ values

In [ ]:
exdf = show_metric_top(ex_trig_abbr,"LRC")
pos_prone, neg_prone = update_prone(exdf, pos_prone, neg_prone)
exdf

LRC        quant     f  unexp_f  dP1  dP2  \
polarity l1         l2                                                     
negative no         longer    9.27    not_exist   742   737.50 0.83 0.08   
                    earthly   7.85    not_exist    74    73.56 0.84 0.01   
                    wise      7.48    not_exist    51    50.71 0.89 0.01   
         never      before    6.30    not_exist   284   266.06 0.92 0.00   
         no         mere      6.10    not_exist    25    24.82 0.69 0.00   
positive or         otherwise 6.45  existential  6378 5,153.93 0.78 0.02   
         everything humanly   6.17    universal   256   248.46 0.74 0.01   
         all        around    5.46    universal   934   792.37 0.79 0.00   
         everything all       5.12    universal  2819 2,686.24 0.46 0.07   
         or         maybe     4.88  existential  2334 1,855.53 0.72 0.01   

                               dP1_simple  dP2_simple        G2   MI  \
polarity l1         l2                                                 
negative no         longer           0.83        0.08  7,109.46 2.22   
                    earthly          0.84        0.01    706.44 2.22   
                    wise             0.89        0.01    501.49 2.25   
         never      before           0.99        0.00  1,535.91 1.20   
         no         mere             0.69        0.00    220.32 2.14   
positive or         otherwise        0.97        0.02 19,728.24 0.72   
         everything humanly          0.76        0.01  1,580.66 1.53   
         all        around           0.93        0.00  3,184.59 0.82   
         everything all              0.48        0.07 13,550.08 1.33   
         or         maybe            0.90        0.01  6,352.68 0.69   

                               odds_r_disc     t        N      f1    f2  \
polarity l1         l2                                                    
negative no         longer            3.03 27.07  1761853    8890   891   
                    earthly           3.01  8.55  1761853    8890    88   
                    wise              3.20  7.10  1761853    8890    57   
         never      before            2.98 15.79  1761853  109723   288   
         no         mere              2.64  4.96  1761853    8890    36   
positive or         otherwise         2.10 64.54  1761853  326614  6603   
         everything humanly           2.15 15.53  1761853   39516   336   
         all        around            1.93 25.93  1761853  249279  1001   
         everything all               1.63 50.59  1761853   39516  5919   
         or         maybe             1.62 38.41  1761853  326614  2581   

                                 exp_f  
polarity l1         l2                  
negative no         longer        4.50  
                    earthly       0.44  
                    wise          0.29  
         never      before       17.94  
         no         mere          0.18  
positive or         otherwise 1,224.07  
         everything humanly       7.54  
         all        around      141.63  
         everything all         132.76  
         or         maybe       478.47

# %% [markdown]

 Top $\Delta P(\texttt{trigger}|\texttt{adv})$ values

In [ ]:
exdf = show_metric_top(ex_trig_abbr, 'dP1')
pos_prone, neg_prone = update_prone(exdf, pos_prone, neg_prone)
exdf

dP1        quant    f  unexp_f  LRC  dP2  \
polarity l1        l2                                                     
negative no        doubt      0.99    not_exist   21    20.89 4.36 0.00   
                   slam       0.97    not_exist   33    32.83 5.08 0.00   
         never     before     0.92    not_exist  284   266.06 6.30 0.00   
         no        wise       0.89    not_exist   51    50.71 7.48 0.01   
         nothing   whatsoever 0.88    not_exist   46    43.25 3.55 0.00   
positive something sizey      0.82  existential   80    65.55 0.98 0.00   
                   primally   0.82  existential   13    10.65 0.00 0.00   
         or        atleast    0.81  existential   34    27.70 0.00 0.00   
                   elsewhere  0.81  existential   24    19.55 0.00 0.00   
         all       around     0.79    universal  934   792.37 5.46 0.00   

                               dP1_simple  dP2_simple       G2   MI  \
polarity l1        l2                                                 
negative no        doubt             1.00        0.00   222.20 2.30   
                   slam              0.97        0.00   340.20 2.28   
         never     before            0.99        0.00 1,535.91 1.20   
         no        wise              0.89        0.01   501.49 2.25   
         nothing   whatsoever        0.94        0.00   242.86 1.22   
positive something sizey             1.00        0.00   273.78 0.74   
                   primally          1.00        0.00    44.49 0.74   
         or        atleast           1.00        0.00   114.61 0.73   
                   elsewhere         1.00        0.00    80.90 0.73   
         all       around            0.93        0.00 3,184.59 0.82   

                               odds_r_disc     t        N      f1    f2  exp_f  
polarity l1        l2                                                           
negative no        doubt              3.93  4.56  1761853    8890    21   0.11  
                   slam               3.65  5.71  1761853    8890    34   0.17  
         never     before             2.98 15.79  1761853  109723   288  17.94  
         no        wise               3.20  7.10  1761853    8890    57   0.29  
         nothing   whatsoever         2.35  6.38  1761853   98788    49   2.75  
positive something sizey              2.86  7.33  1761853  318343    80  14.45  
                   primally           2.09  2.95  1761853  318343    13   2.35  
         or        atleast            2.48  4.75  1761853  326614    34   6.30  
                   elsewhere          2.33  3.99  1761853  326614    24   4.45  
         all       around             1.93 25.93  1761853  249279  1001 141.63

# %% [markdown]

 Top conditional probability $P(\texttt{trigger}|\texttt{adv})$ values

In [ ]:
exdf = show_metric_top(ex_trig_abbr, 'dP1_simple')
pos_prone, neg_prone = update_prone(exdf, pos_prone, neg_prone)
exdf

dP1_simple        quant    f  unexp_f  LRC  \
polarity l1        l2                                                       
negative no        doubt             1.00    not_exist   21    20.89 4.36   
         never     before            0.99    not_exist  284   266.06 6.30   
         no        slam              0.97    not_exist   33    32.83 5.08   
         nothing   whatsoever        0.94    not_exist   46    43.25 3.55   
         no        wise              0.89    not_exist   51    50.71 7.48   
positive something sizey             1.00  existential   80    65.55 0.98   
         or        atleast           1.00  existential   34    27.70 0.00   
         something primally          1.00  existential   13    10.65 0.00   
         or        elsewhere         1.00  existential   24    19.55 0.00   
                   at                0.97  existential  143   115.75 3.50   

                               dP1  dP2  dP2_simple       G2   MI  \
polarity l1        l2                                               
negative no        doubt      0.99 0.00        0.00   222.20 2.30   
         never     before     0.92 0.00        0.00 1,535.91 1.20   
         no        slam       0.97 0.00        0.00   340.20 2.28   
         nothing   whatsoever 0.88 0.00        0.00   242.86 1.22   
         no        wise       0.89 0.01        0.01   501.49 2.25   
positive something sizey      0.82 0.00        0.00   273.78 0.74   
         or        atleast    0.81 0.00        0.00   114.61 0.73   
         something primally   0.82 0.00        0.00    44.49 0.74   
         or        elsewhere  0.81 0.00        0.00    80.90 0.73   
                   at         0.79 0.00        0.00   446.98 0.72   

                               odds_r_disc     t        N      f1   f2  exp_f  
polarity l1        l2                                                          
negative no        doubt              3.93  4.56  1761853    8890   21   0.11  
         never     before             2.98 15.79  1761853  109723  288  17.94  
         no        slam               3.65  5.71  1761853    8890   34   0.17  
         nothing   whatsoever         2.35  6.38  1761853   98788   49   2.75  
         no        wise               3.20  7.10  1761853    8890   57   0.29  
positive something sizey              2.86  7.33  1761853  318343   80  14.45  
         or        atleast            2.48  4.75  1761853  326614   34   6.30  
         something primally           2.09  2.95  1761853  318343   13   2.35  
         or        elsewhere          2.33  3.99  1761853  326614   24   4.45  
                   at                 2.15  9.68  1761853  326614  147  27.25

# %% [markdown]

 Top $\Delta P(\texttt{adv}|\texttt{trigger})$ values

In [ ]:
exdf = show_metric_top(ex_trig_abbr, 'dP2')
pos_prone, neg_prone = update_prone(exdf, pos_prone, neg_prone)
exdf

dP2             quant      f   unexp_f  LRC  dP1  \
polarity l1        l2                                                       
negative nothing   more 0.29         not_exist  43276 27,262.15 1.56 0.11   
         scarcely  more 0.29  hedged_not_exist    354    226.75 1.18 0.00   
                   less 0.26  hedged_not_exist    216    204.52 3.81 0.01   
         no        less 0.23         not_exist   2194  2,063.96 4.05 0.08   
         barely    even 0.23  hedged_not_exist    324    281.29 2.57 0.00   
positive either    too  0.20       existential   1110    761.33 1.49 0.01   
         everyone  so   0.18         universal   8043  5,665.02 1.75 0.05   
         everybody so   0.16         universal   1894  1,308.09 1.56 0.01   
         always    very 0.15         universal  19988 11,402.98 1.27 0.07   
         or        even 0.13       existential  45543 34,247.18 3.63 0.58   

                         dP1_simple  dP2_simple        G2   MI  odds_r_disc  \
polarity l1        l2                                                         
negative nothing   more        0.15        0.44 45,073.98 0.43         0.66   
         scarcely  more        0.00        0.45    360.21 0.44         0.63   
                   less        0.01        0.28    919.88 1.27         1.41   
         no        less        0.09        0.25  8,973.94 1.23         1.38   
         barely    even        0.01        0.26    824.09 0.88         1.00   
positive either    too         0.01        0.29  1,229.19 0.50         0.61   
         everyone  so          0.06        0.25  9,694.03 0.53         0.64   
         everybody so          0.01        0.24  2,089.53 0.51         0.60   
         always    very        0.11        0.24 13,687.67 0.37         0.47   
         or        even        0.75        0.14 95,651.97 0.61         1.17   

                             t        N      f1      f2     exp_f  
polarity l1        l2                                              
negative nothing   more 131.05  1761853   98788  285602 16,013.85  
         scarcely  more  12.05  1761853     785  285602    127.25  
                   less  13.92  1761853     785   25772     11.48  
         no        less  44.06  1761853    8890   25772    130.04  
         barely    even  15.63  1761853    1235   60933     42.71  
positive either    too   22.85  1761853    3886  158082    348.67  
         everyone  so    63.17  1761853   32327  129602  2,377.98  
         everybody so    30.06  1761853    7965  129602    585.91  
         always    very  80.66  1761853   82200  184009  8,585.02  
         or        even 160.48  1761853  326614   60933 11,295.82

# %% [markdown]

 Top conditional probability $P(\texttt{adv}|\texttt{trigger})$ values

In [ ]:
exdf = show_metric_top(ex_trig_abbr, 'dP2_simple')
pos_prone, neg_prone = update_prone(exdf, pos_prone, neg_prone)
exdf

dP2_simple             quant      f   unexp_f  LRC  \
polarity l1         l2                                                         
negative scarcely   more        0.45  hedged_not_exist    354    226.75 1.18   
         nothing    more        0.44         not_exist  43276 27,262.15 1.56   
         hardly     more        0.36  hedged_not_exist   1944  1,057.62 1.00   
         no         more        0.31         not_exist   2764  1,322.90 0.83   
         scarcely   less        0.28  hedged_not_exist    216    204.52 3.81   
positive either     too         0.29       existential   1110    761.33 1.49   
         everyone   so          0.25         universal   8043  5,665.02 1.75   
         always     very        0.24         universal  19988 11,402.98 1.27   
         everybody  so          0.24         universal   1894  1,308.09 1.56   
         somethings more        0.21       existential     33      7.55 0.00   

                          dP1  dP2  dP1_simple        G2   MI  odds_r_disc  \
polarity l1         l2                                                       
negative scarcely   more 0.00 0.29        0.00    360.21 0.44         0.63   
         nothing    more 0.11 0.29        0.15 45,073.98 0.43         0.66   
         hardly     more 0.00 0.19        0.01  1,208.37 0.34         0.46   
         no         more 0.01 0.15        0.01  1,212.03 0.28         0.37   
         scarcely   less 0.01 0.26        0.01    919.88 1.27         1.41   
positive either     too  0.01 0.20        0.01  1,229.19 0.50         0.61   
         everyone   so   0.05 0.18        0.06  9,694.03 0.53         0.64   
         always     very 0.07 0.15        0.11 13,687.67 0.37         0.47   
         everybody  so   0.01 0.16        0.01  2,089.53 0.51         0.60   
         somethings more 0.00 0.05        0.00      2.49 0.11         0.14   

                              t        N     f1      f2     exp_f  
polarity l1         l2                                             
negative scarcely   more  12.05  1761853    785  285602    127.25  
         nothing    more 131.05  1761853  98788  285602 16,013.85  
         hardly     more  23.99  1761853   5468  285602    886.38  
         no         more  25.16  1761853   8890  285602  1,441.10  
         scarcely   less  13.92  1761853    785   25772     11.48  
positive either     too   22.85  1761853   3886  158082    348.67  
         everyone   so    63.17  1761853  32327  129602  2,377.98  
         always     very  80.66  1761853  82200  184009  8,585.02  
         everybody  so    30.06  1761853   7965  129602    585.91  
         somethings more   1.31  1761853    157  285602     25.45

# %% [markdown]

 Top log-likelihood $G^2$ values

In [ ]:
exdf = show_metric_top(ex_trig_abbr, 'G2')
pos_prone, neg_prone = update_prone(exdf, pos_prone, neg_prone)
exdf

G2        quant      f   unexp_f  LRC  dP1  \
polarity l1      l2                                                             
negative nothing more       45,073.98    not_exist  43276 27,262.15 1.56 0.11   
         never   too        22,017.64    not_exist  25628 15,783.12 1.50 0.11   
                 that        9,132.10    not_exist   2939  2,597.22 3.92 0.47   
         no      less        8,973.94    not_exist   2194  2,063.96 4.05 0.08   
         nothing inherently  8,091.44    not_exist   2565  2,280.44 3.90 0.45   
positive or      even       95,651.97  existential  45543 34,247.18 3.63 0.58   
         all     too        25,133.97    universal  45382 23,015.48 1.25 0.16   
         or      otherwise  19,728.24  existential   6378  5,153.93 6.45 0.78   
                 just       16,976.22  existential  14940  9,680.56 2.20 0.35   
         always  very       13,687.67    universal  19988 11,402.98 1.27 0.07   

                             dP2  dP1_simple  dP2_simple   MI  odds_r_disc  \
polarity l1      l2                                                          
negative nothing more       0.29        0.15        0.44 0.43         0.66   
         never   too        0.15        0.16        0.23 0.42         0.54   
                 that       0.03        0.54        0.03 0.93         1.25   
         no      less       0.23        0.09        0.25 1.23         1.38   
         nothing inherently 0.02        0.51        0.03 0.95         1.25   
positive or      even       0.13        0.75        0.14 0.61         1.17   
         all     too        0.11        0.29        0.18 0.31         0.44   
         or      otherwise  0.02        0.97        0.02 0.72         2.10   
                 just       0.04        0.53        0.05 0.45         0.71   
         always  very       0.15        0.11        0.24 0.37         0.47   

                                 t        N      f1      f2     exp_f  
polarity l1      l2                                                    
negative nothing more       131.05  1761853   98788  285602 16,013.85  
         never   too         98.59  1761853  109723  158082  9,844.88  
                 that        47.91  1761853  109723    5488    341.78  
         no      less        44.06  1761853    8890   25772    130.04  
         nothing inherently  45.03  1761853   98788    5075    284.56  
positive or      even       160.48  1761853  326614   60933 11,295.82  
         all     too        108.04  1761853  249279  158082 22,366.52  
         or      otherwise   64.54  1761853  326614    6603  1,224.07  
                 just        79.20  1761853  326614   28371  5,259.44  
         always  very        80.66  1761853   82200  184009  8,585.02

In [ ]:
# %%

sig_trig_abbr = ex_trig_abbr.loc[ex_trig_abbr.LRC.abs() > 1, :]
def show_prone_ex(amdf, prone_list, list_index):
    try: 
        example = prone[list_index]

    except IndexError: 
        return list_index + 1, f'No {list_index}th entry'
    else:
        print(f'>> {example} <<')
        return list_index+ 1, amdf.filter(like=f'~{example}', axis=0).iloc[:10, :]

# %% [markdown]

 ---
 Significant Examples for Items demonstrated positive polarity environment lean overall (Top 10 by $|LRC|>1$)

In [ ]:
prone = pos_prone[UNIT]
prone

['very',
 'even',
 'just',
 'pretty',
 'rather',
 'slightly',
 'otherwise',
 'fairly',
 'relatively',
 'plain',
 'downright',
 'somewhat',
 'maybe',
 'utterly',
 'definitely',
 'humanly',
 'sizey',
 'primally',
 'atleast',
 'too',
 'so']

In [ ]:
# %%

ix = 0
ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> very <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,MI,odds_r_disc,t,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,,,,
always~very,positive,universal,19988,"11,402.98",1.27,0.07,0.15,0.11,0.24,"13,687.67",0.37,0.47,80.66,1761853,82200,184009,"8,585.02",always,very
none~very,negative,not_exist,1371,"-1,868.85",-1.06,-0.01,-0.06,0.01,0.04,"-1,524.12",-0.37,-0.41,-50.47,1761853,31021,184009,"3,239.85",none,very
rarely~very,negative,hedged_not_exist,133,-330.61,-1.17,-0.00,-0.07,0.00,0.03,-356.50,-0.54,-0.58,-28.67,1761853,4439,184009,463.61,rarely,very
nobody~very,negative,not_exist,158,-501.12,-1.48,-0.00,-0.08,0.00,0.03,-595.60,-0.62,-0.66,-39.87,1761853,6311,184009,659.12,nobody,very
never~very,negative,not_exist,3848,"-7,611.54",-1.52,-0.05,-0.07,0.02,0.04,"-7,769.76",-0.47,-0.53,-122.70,1761853,109723,184009,"11,459.54",never,very
hardly~very,negative,hedged_not_exist,83,-488.08,-1.97,-0.00,-0.09,0.00,0.02,-704.56,-0.84,-0.88,-53.57,1761853,5468,184009,571.08,hardly,very
no~very,negative,not_exist,146,-782.48,-2.06,-0.00,-0.09,0.00,0.02,"-1,103.00",-0.80,-0.84,-64.76,1761853,8890,184009,928.48,no,very
nothing~very,negative,not_exist,1694,"-8,623.48",-2.50,-0.05,-0.09,0.01,0.02,"-12,411.45",-0.78,-0.85,-209.52,1761853,98788,184009,"10,317.48",nothing,very


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> even <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,MI,odds_r_disc,t,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,,,,
or~even,positive,existential,45543,"34,247.18",3.63,0.58,0.13,0.75,0.14,"95,651.97",0.61,1.17,160.48,1761853,326614,60933,"11,295.82",or,even
barely~even,negative,hedged_not_exist,324,281.29,2.57,0.00,0.23,0.01,0.26,824.09,0.88,1.00,15.63,1761853,1235,60933,42.71,barely,even
neither~even,negative,not_exist,53,-170.35,-1.06,-0.00,-0.03,0.00,0.01,-193.33,-0.62,-0.63,-23.40,1761853,6458,60933,223.35,neither,even
no~even,negative,not_exist,74,-233.46,-1.19,-0.00,-0.03,0.00,0.01,-263.35,-0.62,-0.63,-27.14,1761853,8890,60933,307.46,no,even
none~even,negative,not_exist,346,-726.85,-1.25,-0.01,-0.02,0.01,0.01,-697.21,-0.49,-0.51,-39.08,1761853,31021,60933,"1,072.85",none,even
either~even,positive,existential,10,-124.40,-1.39,-0.00,-0.03,0.00,0.00,-201.17,-1.13,-1.12,-39.34,1761853,3886,60933,134.40,either,even
something~even,positive,existential,3863,"-7,146.77",-1.58,-0.12,-0.03,0.06,0.01,"-7,380.93",-0.45,-0.53,-114.99,1761853,318343,60933,"11,009.77",something,even
many~even,positive,hedged_universal,381,"-1,249.08",-1.74,-0.02,-0.03,0.01,0.01,"-1,451.57",-0.63,-0.65,-63.99,1761853,47133,60933,"1,630.08",many,even
someone~even,positive,existential,205,-820.85,-1.82,-0.01,-0.03,0.00,0.01,"-1,016.40",-0.70,-0.72,-57.33,1761853,29662,60933,"1,025.85",someone,even


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> just <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,MI,odds_r_disc,t,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,,,,
or~just,positive,existential,14940,"9,680.56",2.20,0.35,0.04,0.53,0.05,"16,976.22",0.45,0.71,79.20,1761853,326614,28371,"5,259.44",or,just
everything~just,positive,universal,2913,"2,276.68",2.18,0.08,0.06,0.10,0.07,"4,640.62",0.66,0.72,42.18,1761853,39516,28371,636.32,everything,just
many~justifiably,positive,hedged_universal,22,19.65,1.76,0.22,0.00,0.25,0.00,63.95,0.97,1.09,4.19,1761853,47133,88,2.35,many,justifiably
sometimes~just,positive,existential,2596,"1,393.00",1.03,0.05,0.02,0.09,0.03,"1,307.85",0.33,0.37,27.34,1761853,74707,28371,"1,203.00",sometimes,just
someone~just,positive,existential,128,-349.65,-1.26,-0.01,-0.01,0.00,0.00,-370.79,-0.57,-0.58,-30.90,1761853,29662,28371,477.65,someone,just
both~just,positive,universal,238,-567.26,-1.30,-0.02,-0.01,0.01,0.00,-572.63,-0.53,-0.54,-36.77,1761853,50007,28371,805.26,both,just
always~just,positive,universal,397,-926.66,-1.41,-0.03,-0.01,0.01,0.00,-939.66,-0.52,-0.54,-46.51,1761853,82200,28371,"1,323.66",always,just
often~just,positive,hedged_universal,427,"-1,058.22",-1.49,-0.04,-0.01,0.02,0.00,"-1,105.99",-0.54,-0.56,-51.21,1761853,92233,28371,"1,485.22",often,just
nor~just,negative,not_exist,35,-215.34,-1.59,-0.01,-0.01,0.00,0.00,-297.64,-0.85,-0.86,-36.40,1761853,15546,28371,250.34,nor,just


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> pretty <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,MI,odds_r_disc,t,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,,,,
everybody~pretty,positive,universal,421,309.20,1.57,0.01,0.04,0.02,0.05,514.35,0.58,0.60,15.07,1761853,7965,24729,111.80,everybody,pretty
some~pretty,positive,existential,4085,"2,471.88",1.38,0.10,0.02,0.17,0.04,"2,979.78",0.40,0.46,38.68,1761853,114929,24729,"1,613.12",some,pretty
all~pretty,positive,universal,7720,"4,221.17",1.36,0.17,0.02,0.31,0.03,"4,762.72",0.34,0.45,48.04,1761853,249279,24729,"3,498.83",all,pretty
both~pretty,positive,universal,1764,"1,062.11",1.21,0.04,0.02,0.07,0.04,"1,198.43",0.40,0.43,25.29,1761853,50007,24729,701.89,both,pretty
no~pretty,negative,not_exist,18,-106.78,-1.04,-0.00,-0.01,0.00,0.00,-145.62,-0.84,-0.84,-25.17,1761853,8890,24729,124.78,no,pretty
or~pretty,positive,existential,1099,"-3,485.29",-2.06,-0.14,-0.01,0.04,0.00,"-4,448.44",-0.62,-0.70,-105.13,1761853,326614,24729,"4,584.29",or,pretty
nor~pretty,negative,not_exist,11,-207.20,-2.07,-0.01,-0.01,0.00,0.00,-353.24,-1.30,-1.29,-62.47,1761853,15546,24729,218.20,nor,pretty
none~pretty,negative,not_exist,14,-421.40,-2.98,-0.02,-0.01,0.00,0.00,-759.72,-1.49,-1.49,-112.63,1761853,31021,24729,435.40,none,pretty
nothing~pretty,negative,not_exist,46,"-1,340.57",-3.89,-0.05,-0.01,0.00,0.00,"-2,462.81",-1.48,-1.51,-197.66,1761853,98788,24729,"1,386.57",nothing,pretty


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> rather <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,MI,odds_r_disc,t,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,,,,
nothing~rather,negative,not_exist,12,-452.88,-3.20,-0.05,-0.00,0.00,0.00,-845.92,-1.59,-1.60,-130.74,1761853,98788,8291,464.88,nothing,rather


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> slightly <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,MI,odds_r_disc,t,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,,,,
or~slightly,positive,existential,3411,"1,996.36",1.66,0.26,0.01,0.45,0.01,"2,750.08",0.38,0.55,34.18,1761853,326614,7631,"1,414.64",or,slightly
all~slightly,positive,universal,452,-627.69,-1.03,-0.08,-0.00,0.06,0.00,-528.37,-0.38,-0.42,-29.52,1761853,249279,7631,"1,079.69",all,slightly
none~slightly,negative,not_exist,16,-118.36,-1.22,-0.02,-0.00,0.00,0.00,-170.94,-0.92,-0.92,-29.59,1761853,31021,7631,134.36,none,slightly
many~slightly,positive,hedged_universal,34,-170.14,-1.33,-0.02,-0.00,0.00,0.00,-222.90,-0.78,-0.78,-29.18,1761853,47133,7631,204.14,many,slightly
nothing~slightly,negative,not_exist,56,-371.87,-2.00,-0.05,-0.00,0.01,0.00,-536.36,-0.88,-0.90,-49.69,1761853,98788,7631,427.87,nothing,slightly
never~slightly,negative,not_exist,13,-462.24,-3.21,-0.06,-0.00,0.00,0.00,-862.22,-1.56,-1.58,-128.20,1761853,109723,7631,475.24,never,slightly


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> otherwise <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,MI,odds_r_disc,t,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,,,,
or~otherwise,positive,existential,6378,"5,153.93",6.45,0.78,0.02,0.97,0.02,"19,728.24",0.72,2.10,64.54,1761853,326614,6603,"1,224.07",or,otherwise
many~otherwise,positive,hedged_universal,26,-150.64,-1.33,-0.02,-0.00,0.00,0.00,-205.65,-0.83,-0.84,-29.54,1761853,47133,6603,176.64,many,otherwise
often~otherwise,positive,hedged_universal,25,-320.67,-2.36,-0.05,-0.00,0.00,0.00,-527.34,-1.14,-1.16,-64.13,1761853,92233,6603,345.67,often,otherwise
nothing~otherwise,negative,not_exist,15,-355.23,-2.77,-0.05,-0.00,0.00,0.00,-635.51,-1.39,-1.40,-91.72,1761853,98788,6603,370.23,nothing,otherwise
some~otherwise,positive,existential,20,-410.73,-2.85,-0.06,-0.00,0.00,0.00,-726.98,-1.33,-1.35,-91.84,1761853,114929,6603,430.73,some,otherwise
something~otherwise,positive,existential,55,"-1,138.07",-3.70,-0.17,-0.00,0.01,0.00,"-2,166.86",-1.34,-1.42,-153.46,1761853,318343,6603,"1,193.07",something,otherwise
all~otherwise,positive,universal,17,-917.24,-4.18,-0.14,-0.00,0.00,0.00,"-1,843.20",-1.74,-1.79,-222.46,1761853,249279,6603,934.24,all,otherwise


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> fairly <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,MI,odds_r_disc,t,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,,,,
both~fairly,positive,universal,483,321.16,1.31,0.06,0.01,0.08,0.01,435.03,0.47,0.50,14.61,1761853,50007,5702,161.84,both,fairly
some~fairly,positive,existential,882,510.05,1.12,0.09,0.00,0.15,0.01,555.87,0.37,0.42,17.17,1761853,114929,5702,371.95,some,fairly
nothing~fairly,negative,not_exist,10,-309.71,-2.71,-0.05,-0.00,0.00,0.00,-568.65,-1.50,-1.51,-97.94,1761853,98788,5702,319.71,nothing,fairly


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> relatively <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,MI,odds_r_disc,t,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,,,,
both~relatively,positive,universal,465,312.70,1.34,0.06,0.01,0.09,0.01,433.80,0.48,0.51,14.50,1761853,50007,5366,152.30,both,relatively
many~relatively,positive,hedged_universal,418,274.45,1.24,0.05,0.01,0.08,0.01,360.94,0.46,0.49,13.42,1761853,47133,5366,143.55,many,relatively


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> plain <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,MI,odds_r_disc,t,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,,,,
or~plain,positive,existential,3681,"2,741.30",3.31,0.54,0.01,0.73,0.01,"7,053.82",0.59,1.07,45.18,1761853,326614,5069,939.70,or,plain
always~plain,positive,universal,16,-220.50,-2.08,-0.04,-0.00,0.00,0.00,-365.34,-1.17,-1.18,-55.12,1761853,82200,5069,236.50,always,plain
all~plain,positive,universal,67,-650.20,-2.70,-0.13,-0.00,0.01,0.00,"-1,077.34",-1.03,-1.09,-79.43,1761853,249279,5069,717.20,all,plain
something~plain,positive,existential,56,-859.90,-3.30,-0.17,-0.00,0.01,0.00,"-1,576.54",-1.21,-1.29,-114.91,1761853,318343,5069,915.90,something,plain


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> downright <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,MI,odds_r_disc,t,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,,,,
sometimes~downright,positive,existential,1034,832.16,2.39,0.18,0.01,0.22,0.01,"1,886.10",0.71,0.80,25.88,1761853,74707,4760,201.84,sometimes,downright
some~downright,positive,existential,1279,968.50,2.15,0.20,0.01,0.27,0.01,"1,921.01",0.61,0.73,27.08,1761853,114929,4760,310.50,some,downright
or~downright,positive,existential,1779,896.59,1.17,0.19,0.00,0.37,0.01,929.98,0.30,0.42,21.26,1761853,326614,4760,882.41,or,downright
something~downright,positive,existential,132,-728.07,-2.29,-0.15,-0.00,0.03,0.00,"-1,091.55",-0.81,-0.89,-63.37,1761853,318343,4760,860.07,something,downright
all~downright,positive,universal,46,-627.48,-2.97,-0.13,-0.00,0.01,0.00,"-1,101.65",-1.17,-1.22,-92.52,1761853,249279,4760,673.48,all,downright


# %% [markdown]

 ---
## Examples for Items demonstrating *Negative* polarity environment lean overall (Top 10 by $|LRC|>1$)

In [ ]:
prone_list = neg_prone[UNIT]
prone = sig_trig_abbr.copy().loc[sig_trig_abbr.l2.isin(prone_list), ['f2','l2']].drop_duplicates().reset_index(drop=True).set_index('l2').round(1).sort_values(['f2'], ascending=False).index.to_list()
prone

['more',
 'too',
 'particularly',
 'that',
 'inherently',
 'ever',
 'remotely',
 'any',
 'necessarily',
 'exactly',
 'longer',
 'yet',
 'before',
 'earthly',
 'wise',
 'slam',
 'doubt']

In [ ]:
# %%

ix = 0
ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> more <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,MI,odds_r_disc,t,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,,,,
nothing~more,negative,not_exist,43276,"27,262.15",1.56,0.11,0.29,0.15,0.44,"45,073.98",0.43,0.66,131.05,1761853,98788,285602,"16,013.85",nothing,more
scarcely~more,negative,hedged_not_exist,354,226.75,1.18,0.00,0.29,0.00,0.45,360.21,0.44,0.63,12.05,1761853,785,285602,127.25,scarcely,more
hardly~more,negative,hedged_not_exist,1944,"1,057.62",1.00,0.00,0.19,0.01,0.36,"1,208.37",0.34,0.46,23.99,1761853,5468,285602,886.38,hardly,more
either~more,positive,existential,180,-449.93,-1.26,-0.00,-0.12,0.00,0.05,-509.25,-0.54,-0.60,-33.54,1761853,3886,285602,629.93,either,more
both~more,positive,universal,2929,"-5,177.29",-1.36,-0.02,-0.11,0.01,0.06,"-5,120.85",-0.44,-0.50,-95.66,1761853,50007,285602,"8,106.29",both,more
many~more,positive,hedged_universal,2605,"-5,035.41",-1.44,-0.02,-0.11,0.01,0.06,"-5,189.46",-0.47,-0.53,-98.66,1761853,47133,285602,"7,640.41",many,more
everything~more,positive,universal,2144,"-4,261.67",-1.44,-0.02,-0.11,0.01,0.05,"-4,433.80",-0.48,-0.54,-92.04,1761853,39516,285602,"6,405.67",everything,more
nor~more,negative,not_exist,747,"-1,773.06",-1.50,-0.01,-0.12,0.00,0.05,"-1,973.77",-0.53,-0.59,-64.87,1761853,15546,285602,"2,520.06",nor,more
everybody~more,positive,universal,336,-955.15,-1.55,-0.00,-0.12,0.00,0.04,"-1,140.11",-0.58,-0.64,-52.11,1761853,7965,285602,"1,291.15",everybody,more


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> too <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,MI,odds_r_disc,t,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,,,,
never~too,negative,not_exist,25628,"15,783.12",1.50,0.11,0.15,0.16,0.23,"22,017.64",0.42,0.54,98.59,1761853,109723,158082,"9,844.88",never,too
either~too,positive,existential,1110,761.33,1.49,0.01,0.20,0.01,0.29,"1,229.19",0.50,0.61,22.85,1761853,3886,158082,348.67,either,too
all~too,positive,universal,45382,"23,015.48",1.25,0.16,0.11,0.29,0.18,"25,133.97",0.31,0.44,108.04,1761853,249279,158082,"22,366.52",all,too
somebody~too,positive,existential,62,-178.64,-1.02,-0.00,-0.07,0.00,0.02,-202.10,-0.59,-0.62,-22.69,1761853,2682,158082,240.64,somebody,too
always~too,positive,universal,3374,"-4,001.38",-1.04,-0.03,-0.05,0.02,0.04,"-3,051.56",-0.34,-0.37,-68.89,1761853,82200,158082,"7,375.38",always,too
rarely~too,negative,hedged_not_exist,115,-283.29,-1.11,-0.00,-0.06,0.00,0.03,-300.83,-0.54,-0.57,-26.42,1761853,4439,158082,398.29,rarely,too
some~too,positive,existential,4285,"-6,026.99",-1.21,-0.04,-0.06,0.03,0.04,"-5,135.45",-0.38,-0.42,-92.07,1761853,114929,158082,"10,311.99",some,too
everybody~too,positive,universal,219,-495.66,-1.21,-0.00,-0.06,0.00,0.03,-508.13,-0.51,-0.54,-33.49,1761853,7965,158082,714.66,everybody,too
everyone~too,positive,universal,1046,"-1,854.54",-1.26,-0.01,-0.06,0.01,0.03,"-1,714.15",-0.44,-0.47,-57.34,1761853,32327,158082,"2,900.54",everyone,too


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> particularly <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,MI,odds_r_disc,t,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,,,,
neither~particularly,negative,not_exist,815,767.20,3.93,0.06,0.12,0.06,0.13,"3,230.78",1.23,1.31,26.87,1761853,6458,13041,47.80,neither,particularly
none~particularly,negative,not_exist,2028,"1,798.39",3.19,0.14,0.06,0.16,0.07,"5,613.04",0.95,1.04,39.93,1761853,31021,13041,229.61,none,particularly
nor~particularly,negative,not_exist,979,863.93,2.95,0.07,0.06,0.08,0.06,"2,573.01",0.93,0.99,27.61,1761853,15546,13041,115.07,nor,particularly
nothing~particularly,negative,not_exist,3686,"2,954.78",2.59,0.23,0.03,0.28,0.04,"6,884.51",0.70,0.84,48.67,1761853,98788,13041,731.22,nothing,particularly
often~particularly,positive,hedged_universal,216,-466.70,-1.20,-0.04,-0.01,0.02,0.00,-456.17,-0.50,-0.52,-31.75,1761853,92233,13041,682.70,often,particularly
or~particularly,positive,existential,912,"-1,505.55",-1.34,-0.12,-0.01,0.07,0.00,"-1,445.46",-0.42,-0.48,-49.85,1761853,326614,13041,"2,417.55",or,particularly
everyone~particularly,positive,universal,15,-224.28,-2.09,-0.02,-0.01,0.00,0.00,-370.97,-1.20,-1.20,-57.91,1761853,32327,13041,239.28,everyone,particularly
sometimes~particularly,positive,existential,58,-494.97,-2.33,-0.04,-0.01,0.00,0.00,-751.19,-0.98,-1.00,-64.99,1761853,74707,13041,552.97,sometimes,particularly
everything~particularly,positive,universal,14,-278.49,-2.42,-0.02,-0.01,0.00,0.00,-479.94,-1.32,-1.32,-74.43,1761853,39516,13041,292.49,everything,particularly


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> that <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,MI,odds_r_disc,t,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,,,,
never~that,negative,not_exist,2939,"2,597.22",3.92,0.47,0.03,0.54,0.03,"9,132.10",0.93,1.25,47.91,1761853,109723,5488,341.78,never,that
rarely~that,negative,hedged_not_exist,194,180.17,3.32,0.03,0.04,0.04,0.04,677.90,1.15,1.18,12.94,1761853,4439,5488,13.83,rarely,that
nobody~that,negative,not_exist,165,145.34,2.52,0.03,0.02,0.03,0.03,418.67,0.92,0.95,11.31,1761853,6311,5488,19.66,nobody,that
none~that,negative,not_exist,354,257.37,1.54,0.05,0.01,0.06,0.01,419.22,0.56,0.59,13.68,1761853,31021,5488,96.63,none,that
neither~that,negative,not_exist,91,70.88,1.41,0.01,0.01,0.02,0.01,134.64,0.66,0.67,7.43,1761853,6458,5488,20.12,neither,that
something~thats,positive,existential,67,47.49,1.41,0.44,0.00,0.62,0.00,102.22,0.54,0.87,5.80,1761853,318343,108,19.51,something,thats
many~that,positive,hedged_universal,20,-126.81,-1.23,-0.02,-0.00,0.00,0.00,-177.23,-0.87,-0.87,-28.36,1761853,47133,5488,146.81,many,that
or~that,positive,existential,318,-699.37,-1.46,-0.13,-0.00,0.06,0.00,-765.07,-0.51,-0.57,-39.22,1761853,326614,5488,"1,017.37",or,that
both~that,positive,universal,13,-142.77,-1.54,-0.03,-0.00,0.00,0.00,-225.17,-1.08,-1.07,-39.60,1761853,50007,5488,155.77,both,that


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> inherently <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,MI,odds_r_disc,t,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,,,,
nothing~inherently,negative,not_exist,2565,"2,280.44",3.90,0.45,0.02,0.51,0.03,"8,091.44",0.95,1.25,45.03,1761853,98788,5075,284.56,nothing,inherently
often~inherently,positive,hedged_universal,40,-225.68,-1.61,-0.04,-0.00,0.01,0.00,-310.89,-0.82,-0.84,-35.68,1761853,92233,5075,265.68,often,inherently
some~inherently,positive,existential,48,-283.05,-1.78,-0.06,-0.00,0.01,0.00,-398.03,-0.84,-0.86,-40.86,1761853,114929,5075,331.05,some,inherently
or~inherently,positive,existential,219,-721.81,-1.82,-0.14,-0.00,0.04,0.00,-926.39,-0.63,-0.70,-48.78,1761853,326614,5075,940.81,or,inherently
all~inherently,positive,universal,130,-588.05,-1.98,-0.12,-0.00,0.03,0.00,-809.39,-0.74,-0.80,-51.58,1761853,249279,5075,718.05,all,inherently
never~inherently,negative,not_exist,19,-297.06,-2.42,-0.06,-0.00,0.00,0.00,-506.30,-1.22,-1.24,-68.15,1761853,109723,5075,316.06,never,inherently


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> ever <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,MI,odds_r_disc,t,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,,,,
hardly~ever,negative,hedged_not_exist,690,674.40,5.39,0.13,0.12,0.14,0.13,"4,062.94",1.65,1.77,25.67,1761853,5468,5027,15.60,hardly,ever
rarely~ever,negative,hedged_not_exist,332,319.33,4.40,0.06,0.07,0.07,0.07,"1,574.57",1.42,1.48,17.53,1761853,4439,5027,12.67,rarely,ever
nothing~ever,negative,not_exist,2102,"1,820.13",3.38,0.36,0.02,0.42,0.02,"5,652.18",0.87,1.09,39.70,1761853,98788,5027,281.87,nothing,ever
nobody~ever,negative,not_exist,206,187.99,3.05,0.04,0.03,0.04,0.03,640.95,1.06,1.09,13.10,1761853,6311,5027,18.01,nobody,ever
seldom~ever,negative,hedged_not_exist,27,24.04,1.77,0.00,0.02,0.01,0.03,71.95,0.96,0.98,4.63,1761853,1038,5027,2.96,seldom,ever
nor~ever,negative,not_exist,181,136.64,1.51,0.03,0.01,0.04,0.01,240.77,0.61,0.63,10.16,1761853,15546,5027,44.36,nor,ever
none~ever,negative,not_exist,285,196.49,1.29,0.04,0.01,0.06,0.01,282.76,0.51,0.53,11.64,1761853,31021,5027,88.51,none,ever
barely~ever,negative,hedged_not_exist,24,20.48,1.26,0.00,0.02,0.00,0.02,51.56,0.83,0.85,4.18,1761853,1235,5027,3.52,barely,ever
never~ever,negative,not_exist,767,453.93,1.15,0.09,0.00,0.15,0.01,513.90,0.39,0.44,16.39,1761853,109723,5027,313.07,never,ever


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> remotely <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,MI,odds_r_disc,t,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,,,,
nothing~remotely,negative,not_exist,1167,"1,036.02",3.76,0.44,0.01,0.50,0.01,"3,632.69",0.95,1.23,30.33,1761853,98788,2336,130.98,nothing,remotely
none~remotely,negative,not_exist,362,320.87,2.93,0.14,0.01,0.15,0.01,983.40,0.94,1.02,16.86,1761853,31021,2336,41.13,none,remotely
neither~remotely,negative,not_exist,67,58.44,2.08,0.03,0.01,0.03,0.01,160.82,0.89,0.91,7.14,1761853,6458,2336,8.56,neither,remotely
nor~remotely,negative,not_exist,76,55.39,1.05,0.02,0.00,0.03,0.00,89.10,0.57,0.58,6.35,1761853,15546,2336,20.61,nor,remotely
or~remotely,positive,existential,135,-298.05,-1.23,-0.13,-0.00,0.06,0.00,-326.15,-0.51,-0.57,-25.65,1761853,326614,2336,433.05,or,remotely
all~remotely,positive,universal,17,-313.51,-2.67,-0.13,-0.00,0.01,0.00,-573.23,-1.29,-1.34,-76.04,1761853,249279,2336,330.51,all,remotely


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> any <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,MI,odds_r_disc,t,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,,,,
hardly~any,negative,hedged_not_exist,244,240.34,5.86,0.20,0.04,0.21,0.04,"1,633.00",1.82,1.94,15.39,1761853,5468,1178,3.66,hardly,any
barely~any,negative,hedged_not_exist,66,65.17,5.48,0.06,0.05,0.06,0.05,455.14,1.90,1.95,8.02,1761853,1235,1178,0.83,barely,any
scarcely~any,negative,hedged_not_exist,34,33.48,4.78,0.03,0.04,0.03,0.04,219.09,1.81,1.85,5.74,1761853,785,1178,0.52,scarcely,any
nor~any,negative,not_exist,103,92.61,2.66,0.08,0.01,0.09,0.01,295.35,1.00,1.04,9.12,1761853,15546,1178,10.39,nor,any
nobody~any,negative,not_exist,48,43.78,2.46,0.04,0.01,0.04,0.01,147.82,1.06,1.08,6.32,1761853,6311,1178,4.22,nobody,any
none~any,negative,not_exist,145,124.26,2.31,0.11,0.00,0.12,0.00,329.78,0.84,0.90,10.32,1761853,31021,1178,20.74,none,any
never~any,negative,not_exist,283,209.64,1.74,0.18,0.00,0.24,0.00,387.84,0.59,0.68,12.46,1761853,109723,1178,73.36,never,any
none~anywhere,negative,not_exist,10,9.30,1.49,0.23,0.00,0.25,0.00,36.87,1.15,1.28,2.94,1761853,31021,40,0.70,none,anywhere
neither~any,negative,not_exist,26,21.68,1.14,0.02,0.00,0.02,0.00,50.47,0.78,0.80,4.25,1761853,6458,1178,4.32,neither,any


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> necessarily <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,MI,odds_r_disc,t,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,,,,
nor~necessarily,negative,not_exist,264,254.29,4.62,0.23,0.02,0.24,0.02,"1,304.27",1.43,1.56,15.65,1761853,15546,1100,9.71,nor,necessarily
neither~necessarily,negative,not_exist,96,91.97,3.91,0.08,0.01,0.09,0.01,433.98,1.38,1.42,9.39,1761853,6458,1100,4.03,neither,necessarily
none~necessarily,negative,not_exist,222,202.63,3.26,0.18,0.01,0.20,0.01,719.66,1.06,1.15,13.60,1761853,31021,1100,19.37,none,necessarily
nothing~necessarily,negative,not_exist,344,282.32,2.45,0.26,0.00,0.31,0.00,703.55,0.75,0.89,15.22,1761853,98788,1100,61.68,nothing,necessarily
or~necessarily,positive,existential,32,-171.92,-1.58,-0.16,-0.00,0.03,0.00,-256.48,-0.80,-0.87,-30.39,1761853,326614,1100,203.92,or,necessarily
all~necessarily,positive,universal,15,-140.64,-1.63,-0.13,-0.00,0.01,0.00,-231.15,-1.02,-1.06,-36.31,1761853,249279,1100,155.64,all,necessarily
something~necessarily,positive,existential,21,-177.76,-1.85,-0.16,-0.00,0.02,0.00,-294.19,-0.98,-1.04,-38.79,1761853,318343,1100,198.76,something,necessarily


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> exactly <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,MI,odds_r_disc,t,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,,,,
nobody~exactly,negative,not_exist,84,80.30,3.77,0.08,0.01,0.08,0.01,371.27,1.36,1.40,8.76,1761853,6311,1034,3.70,nobody,exactly
neither~exactly,negative,not_exist,73,69.21,3.46,0.07,0.01,0.07,0.01,298.96,1.28,1.32,8.10,1761853,6458,1034,3.79,neither,exactly
none~exactly,negative,not_exist,195,176.79,3.10,0.17,0.01,0.19,0.01,604.96,1.03,1.12,12.66,1761853,31021,1034,18.21,none,exactly
never~exactly,negative,not_exist,325,260.61,2.29,0.25,0.00,0.31,0.00,609.00,0.70,0.84,14.46,1761853,109723,1034,64.39,never,exactly
nor~exactly,negative,not_exist,44,34.88,1.17,0.03,0.00,0.04,0.00,69.98,0.68,0.70,5.26,1761853,15546,1034,9.12,nor,exactly
or~exactly,positive,existential,30,-161.68,-1.54,-0.16,-0.00,0.03,0.00,-241.41,-0.81,-0.88,-29.52,1761853,326614,1034,191.68,or,exactly


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> longer <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,MI,odds_r_disc,t,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,,,,
no~longer,negative,not_exist,742,737.50,9.27,0.83,0.08,0.83,0.08,"7,109.46",2.22,3.03,27.07,1761853,8890,891,4.50,no,longer
something~longer,positive,existential,17,-143.99,-1.67,-0.16,-0.00,0.02,0.00,-238.33,-0.98,-1.04,-34.92,1761853,318343,891,160.99,something,longer


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> yet <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,MI,odds_r_disc,t,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,,,,
none~yet,negative,not_exist,99,84.74,2.16,0.10,0.00,0.12,0.00,223.76,0.84,0.89,8.52,1761853,31021,810,14.26,none,yet
neither~yet,negative,not_exist,32,29.03,2.14,0.04,0.00,0.04,0.00,95.29,1.03,1.06,5.13,1761853,6458,810,2.97,neither,yet
nobody~yet,negative,not_exist,27,24.10,1.80,0.03,0.00,0.03,0.00,73.08,0.97,0.99,4.64,1761853,6311,810,2.90,nobody,yet
nor~yet,negative,not_exist,47,39.85,1.67,0.05,0.00,0.06,0.00,99.45,0.82,0.85,5.81,1761853,15546,810,7.15,nor,yet
